# Steem Mentions

In [123]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import math
from decimal import Decimal
import tabulate
from constants import MONGO_URL_DATA_FILE_PATH, MONGO_URL_COLUMNS, AUTHOR_DATA_FILE_PATH
MONGO_URL_DATA_FILE_PATH

'/home/ubuntu/steem-mentions/./data/mongo_url_list.csv'

## Loading Data

### Url data

In [124]:
url_data = pd.DataFrame.from_csv(MONGO_URL_DATA_FILE_PATH)

In [125]:
url_data.head()

,net_votes,total_payout_value,created,url,host_name,domain,author,parent_author,parent_permlink,permlink
id,,,,,,,,,,
4,6,1.024,2016-04-06 19:54:12,https://youtu.be/_NgFIj9dBkQ,youtu.be,youtu.be,red,NaN,daily-decrypt,red-dailydecrypt-1
19,23,81.774,2016-04-08 15:32:57,https://github.com/wackou,github.com,github.com,wackou,NaN,witness-category,wackou-witness-post
19,23,81.774,2016-04-08 15:32:57,https://github.com/wackou/bts_tools,github.com,github.com,wackou,NaN,witness-category,wackou-witness-post
19,23,81.774,2016-04-08 15:32:57,https://bts-tools.readthedocs.io/,bts-tools.readthedocs.io,readthedocs.io,wackou,NaN,witness-category,wackou-witness-post
19,23,81.774,2016-04-08 15:32:57,https://steemit.com/steem/@wackou/btstools-rel...,steemit.com,steemit.com,wackou,NaN,witness-category,wackou-witness-post


In [126]:
len(url_data)

10156778

### Author data

In [127]:
author_data = pd.DataFrame.from_csv(AUTHOR_DATA_FILE_PATH)
author_data.head()

,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests
name,,,,,,,
itcanbeprevented,2016-08-13 22:59:00,2016-09-05 22:48:48,2016-09-05 22:48:48,71465443,0.000,0.000,1.090607e+04
shootit,2017-07-17 17:10:48,2017-08-19 16:57:57,2017-08-19 16:56:33,68361015,0.000,0.000,1.033498e+03
pratibha,2017-08-09 07:30:09,2017-09-26 15:55:21,2017-09-19 20:22:51,7121429731,0.005,0.000,3.514357e+03
lilytran,2016-07-23 05:01:27,2016-08-17 21:54:42,2016-08-18 20:54:15,8370720214,0.000,0.123,1.304985e+04
exyle,2016-07-14 07:08:00,2017-09-30 18:33:33,2017-09-30 18:27:15,115863519050039,0.000,2578.350,1.861507e+08


Calculate reputation score
https://steemit.com/steemit/@digitalnotvir/how-reputation-scores-are-calculated-the-details-explained-with-simple-math

In [128]:
def reputation_score(reputation):
    neg = False if reputation >= 0 else True
    if reputation == 0:
        out = 0
    else:
        out = math.log(math.fabs(reputation), 10)
    out = max(out - 9, 0)
    out = out * -1 if neg else out
    out = (out * 9) + 25
    return math.floor(out)

In [129]:
author_data.iloc[39]

created           2017-08-02 08:09:30
last_vote_time    2017-09-15 13:25:24
last_post         2017-08-24 19:30:15
reputation                14520822935
total_steem                     0.001
total_sbd                        4.56
total_vests                   19548.4
Name: geopoliticsnow, dtype: object

In [130]:
reputation_score(author_data.iloc[39]['reputation'])

35

In [131]:
author_data.loc["leyroft"]

created           2017-08-05 04:49:06
last_vote_time    2017-09-30 13:46:57
last_post         2017-08-15 06:08:12
reputation                          0
total_steem                         0
total_sbd                       0.001
total_vests                   1327.21
Name: leyroft, dtype: object

In [132]:
author_data['reputation_score'] = author_data.apply(lambda row: reputation_score(row['reputation']), axis=1)

Add a numerical index for authors

In [133]:
author_data.reset_index(inplace=True)

In [134]:
author_data.head()

,name,created,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
0,itcanbeprevented,2016-08-13 22:59:00,2016-09-05 22:48:48,2016-09-05 22:48:48,71465443,0.000,0.000,1.090607e+04,25
1,shootit,2017-07-17 17:10:48,2017-08-19 16:57:57,2017-08-19 16:56:33,68361015,0.000,0.000,1.033498e+03,25
2,pratibha,2017-08-09 07:30:09,2017-09-26 15:55:21,2017-09-19 20:22:51,7121429731,0.005,0.000,3.514357e+03,32
3,lilytran,2016-07-23 05:01:27,2016-08-17 21:54:42,2016-08-18 20:54:15,8370720214,0.000,0.123,1.304985e+04,33
4,exyle,2016-07-14 07:08:00,2017-09-30 18:33:33,2017-09-30 18:27:15,115863519050039,0.000,2578.350,1.861507e+08,70


### Urls with author data

In [135]:
url_with_author_data = pd.merge(url_data, author_data, left_on='author', right_on="name")
url_with_author_data.head()

,net_votes,total_payout_value,created_x,url,host_name,domain,author,parent_author,parent_permlink,permlink,name,created_y,last_vote_time,last_post,reputation,total_steem,total_sbd,total_vests,reputation_score
0,6,1.024,2016-04-06 19:54:12,https://youtu.be/_NgFIj9dBkQ,youtu.be,youtu.be,red,NaN,daily-decrypt,red-dailydecrypt-1,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
1,-1,0.000,2016-04-12 20:13:18,http://www.washingtonpost.com/wp-srv/national/...,www.washingtonpost.com,washingtonpost.com,red,NaN,spam,spamtford,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
2,0,0.000,2016-04-12 20:54:24,https://www.youtube.com/watch?v=B0N1gQXETmU,www.youtube.com,youtube.com,red,NaN,Does voting even matter?,newswearechange,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
3,0,0.000,2016-04-16 02:58:42,http://blog.tenthamendmentcenter.com/2016/04/s...,blog.tenthamendmentcenter.com,tenthamendmentcenter.com,red,NaN,news,governor-mississippi-constitutional-carry-law-...,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54
4,1,0.184,2016-04-16 03:33:39,https://scontent-ord1-1.xx.fbcdn.net/hphotos-x...,scontent-ord1-1.xx.fbcdn.net,fbcdn.net,red,NaN,news,laughable,red,2016-04-04 03:25:39,2017-06-07 10:15:27,2017-06-07 10:11:39,1993860928264,39.065,53.122,1.632901e+07,54


### Truncate settings

Truncate the url data for testing

In [136]:
use_truncate = True
if use_truncate:
    url_with_author_data = url_with_author_data[0:5000]
    url_data = url_data[0:5000]

### Group by with metrics

In [144]:
def group_on_field_with_metrics(field_name):
    url_authors = url_data.groupby(field_name, as_index=False)['author'].agg({'mentions': lambda x: len(x), 'num_authors': lambda x: len(np.unique(x)),'authors': lambda x: np.unique(x)})
    url_votes = url_data.groupby(field_name, as_index=False)['net_votes'].agg({'total_votes': lambda x: np.sum(x), 'avg_votes': lambda x: np.sum(x) / len(x)})
    url_payouts = url_data.groupby(field_name, as_index=False)['total_payout_value'].agg({'total_payout': lambda x: np.sum(x), 'avg_payout': lambda x: np.sum(x) / len(x)})
    url_reputation = url_with_author_data.groupby(field_name, as_index=False)['reputation_score'].agg({'total_reputation': lambda x: np.sum(x), 'avg_reputation': lambda x: np.sum(x) / len(x)})
    merge_left = pd.merge(url_authors, url_votes)
    merge_left = pd.merge(merge_left, url_payouts)
    unique_totals = pd.merge(merge_left, url_reputation)
    return unique_totals

### Unique urls with metrics data

Let's make a totals data frame for unique urls

In [155]:
unique_urls = group_on_field_with_metrics('url')
# add domain
url_domains = url_data.groupby('url', as_index=False)['domain'].agg({'domain': lambda x: np.unique(x)})
unique_urls = pd.merge(unique_urls, url_domains)
unique_urls.head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation,domain
0,http://127.0.0.1:8090/rpc,xeroc,1,1,29.0,29,2.165,2.165,70,70.0,127.0.0.1.
1,http://41.media.tumblr.com/871e601974eb2366979...,puppies,1,1,0.0,0,0.000,0.000,51,51.0,tumblr.com
2,http://akasha.world,au1nethyb1,1,1,40.0,40,163.136,163.136,62,62.0,akasha.world
3,http://altexplorer.xyz/steempaperwallet.html,penambang,1,1,12.0,12,13.974,13.974,47,47.0,altexplorer.xyz
4,http://arstechnica.com/science/2016/04/hospita...,jabbasteem2,1,1,1.0,1,0.000,0.000,5,5.0,arstechnica.com


### Unique domains with metric data

In [146]:
unique_domains = group_on_field_with_metrics('domain')
unique_domains.head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
0,127.0.0.1.,xeroc,1,1,29.0,29,2.165,2.165,278,69.5
1,500px.com,pfunk,1,1,9.0,9,23.882,23.882,63,63.0
2,SteemServices.com,steemservices,1,1,77.0,77,637.063,637.063,65,65.0
3,aa.com,steempower,1,1,1.0,1,0.000,0.000,72,72.0
4,akamaihd.net,abuali,1,1,-1.0,-1,0.000,0.000,98,49.0


## URL's

### Most mentioned urls

In [103]:
unique_urls.sort_values('mentions', ascending=False).head()

,url,authors,mentions,num_authors,total_payout,avg_payout
3689,https://steemit.com/steemhelp/@dele-puppy/regi...,dele-puppy,8,1,69.372,8.671500
3587,https://steemit.com/,"[complexring, ihashfury, jamesstroud, steemed]",7,4,1886.547,269.506714
2211,https://bitshares.openledger.info/?r=infobot#/...,tuck-fheman,6,1,42.398,7.066333
426,http://gavinandresen.ninja/satoshi,"[au1nethyb1, complexring, cryptorune, tuck-fhe...",5,4,129.526,25.905200
2375,https://daohub.org/,"[cryptorune, democracy, ethereum, germanaure, ...",5,5,0.424,0.084800


### Top URL's by number  authors

In [113]:
unique_urls.sort_values('num_authors', ascending=False).head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout
2375,https://daohub.org/,"[cryptorune, democracy, ethereum, germanaure, ...",5,5,2.60,13,0.424,0.08480
426,http://gavinandresen.ninja/satoshi,"[au1nethyb1, complexring, cryptorune, tuck-fhe...",5,4,6.60,33,129.526,25.90520
1921,http://www.wsj.com/articles/chiefless-company-...,"[hbhades, jupiter00000, mining-at-work, offici...",4,4,9.00,36,125.611,31.40275
1279,http://steem.herokuapp.com,"[abuali, joseph, ned, nextgencrypto]",4,4,62.00,248,210.127,52.53175
1514,http://www.coindesk.com/coinbase-exchange-rebr...,"[bob-the-poster, cryptorune, germanaure, highb...",4,4,3.75,15,0.198,0.04950


### Top URL's by total votes

In [110]:
unique_urls.sort_values('total_votes', ascending=False).head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout
120,http://blocktrades.us,"[blocktrades, tuck-fheman]",4,2,90.250000,361,3433.932,858.483000
2374,https://daohub.org,"[craig-grant, dan, tuck-fheman]",3,3,98.333333,295,1416.160,472.053333
2400,https://download.slock.it/public/DAO/WhitePape...,dan,1,1,292.000000,292,1395.878,1395.878000
2139,https://bitcoinmagazine.com/articles/bootstrap...,dan,1,1,292.000000,292,1395.878,1395.878000
1680,http://www.internetslang.com/DOA-meaning-defin...,dan,1,1,292.000000,292,1395.878,1395.878000


### Top URL's by avg votes with minimum 10 mentions

In [118]:
unique_urls.query('mentions >= 10').sort_values('avg_votes', ascending=False).head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout


### Top URL's by total payout

In [111]:
unique_urls.sort_values('total_payout', ascending=False).head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout
120,http://blocktrades.us,"[blocktrades, tuck-fheman]",4,2,90.250000,361,3433.932,858.483000
3587,https://steemit.com/,"[complexring, ihashfury, jamesstroud, steemed]",7,4,35.428571,248,1886.547,269.506714
938,http://martinfowler.com/articles/lmax.html,"[liondani, peerplays]",2,2,64.500000,129,1701.929,850.964500
30,http://BitShares-Munich.de,"[kencode, tuck-fheman]",3,2,61.666667,185,1658.405,552.801667
3413,https://pevo.science,pharesim,2,1,89.000000,178,1474.897,737.448500


### Top URL's by avg payout with minimum 10 mentions

In [119]:
unique_urls.query('mentions >= 10').sort_values('avg_payout', ascending=False).head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout


## Websites

### Most mentioned

In [147]:
unique_domains.sort_values('mentions', ascending=False).head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
250,youtube.com,"[acidsun, acoustickitty, adambaked, alex90342f...",464,86,4.297414,1994,2990.058,6.444091,7624,65.162393
90,github.com,"[abit, arhag, ash, au1nethyb1, betax, carlos-c...",443,47,14.632054,6482,25646.108,57.891892,17581,67.360153
115,imgur.com,"[abit, addisdev, alex90342fastn1, andzzz, arai...",322,66,5.465839,1760,5740.273,17.826935,31934,67.944681
196,steemit.com,"[abit, arhag, ash, atomrigs, au1nethyb1, benjo...",181,54,19.712707,3568,20952.207,115.758050,46634,68.680412
114,imgsafe.org,"[acoustickitty, anon, ash, bleepcoin, boatymcb...",139,21,13.669065,1900,12524.879,90.107043,6720,67.200000


### Most authors

In [148]:
unique_domains.sort_values('num_authors', ascending=False).head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
250,youtube.com,"[acidsun, acoustickitty, adambaked, alex90342f...",464,86,4.297414,1994,2990.058,6.444091,7624,65.162393
115,imgur.com,"[abit, addisdev, alex90342fastn1, andzzz, arai...",322,66,5.465839,1760,5740.273,17.826935,31934,67.944681
196,steemit.com,"[abit, arhag, ash, atomrigs, au1nethyb1, benjo...",181,54,19.712707,3568,20952.207,115.758050,46634,68.680412
90,github.com,"[abit, arhag, ash, au1nethyb1, betax, carlos-c...",443,47,14.632054,6482,25646.108,57.891892,17581,67.360153
249,youtu.be,"[ceo2537, clains, clayop, coin, curiousmind, d...",130,34,3.546154,461,956.860,7.360462,2631,65.775000


### Most votes

In [149]:
unique_domains.sort_values('total_votes', ascending=False).head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
90,github.com,"[abit, arhag, ash, au1nethyb1, betax, carlos-c...",443,47,14.632054,6482,25646.108,57.891892,17581,67.360153
196,steemit.com,"[abit, arhag, ash, atomrigs, au1nethyb1, benjo...",181,54,19.712707,3568,20952.207,115.758050,46634,68.680412
98,googleusercontent.com,"[bangking, insomnia, pairmike, riverhead]",51,4,40.784314,2080,31816.348,623.849961,289,72.250000
250,youtube.com,"[acidsun, acoustickitty, adambaked, alex90342f...",464,86,4.297414,1994,2990.058,6.444091,7624,65.162393
114,imgsafe.org,"[acoustickitty, anon, ash, bleepcoin, boatymcb...",139,21,13.669065,1900,12524.879,90.107043,6720,67.200000


### Avg vote with with 10 or more mentions

In [150]:
unique_domains.query('mentions >= 10').sort_values('avg_votes', ascending=False).head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
98,googleusercontent.com,"[bangking, insomnia, pairmike, riverhead]",51,4,40.784314,2080,31816.348,623.849961,289,72.250000
169,puu.sh,"[au1nethyb1, danknugs, justin, nextgencrypto, ...",31,5,37.032258,1148,6225.434,200.820452,4221,63.954545
31,bitshares.org,"[chris4210, ihashfury, peerplays, tuck-fheman,...",15,6,36.800000,552,5301.370,353.424667,2199,68.718750
119,ipfs.pics,"[ash, kencode, lafona, lovelace, markopaasila,...",28,10,31.535714,883,6479.172,231.399000,1169,58.450000
172,readthedocs.io,"[au1nethyb1, wackou, xeroc]",11,3,28.454545,313,1376.522,125.138364,1420,67.619048


### Total payout

In [151]:
unique_domains.sort_values('total_payout', ascending=False).head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
98,googleusercontent.com,"[bangking, insomnia, pairmike, riverhead]",51,4,40.784314,2080,31816.348,623.849961,289,72.250000
90,github.com,"[abit, arhag, ash, au1nethyb1, betax, carlos-c...",443,47,14.632054,6482,25646.108,57.891892,17581,67.360153
196,steemit.com,"[abit, arhag, ash, atomrigs, au1nethyb1, benjo...",181,54,19.712707,3568,20952.207,115.758050,46634,68.680412
96,google.com,"[au1nethyb1, cass, chris4210, cnavigato, dedri...",36,19,27.472222,989,14466.093,401.835917,1962,67.655172
114,imgsafe.org,"[acoustickitty, anon, ash, bleepcoin, boatymcb...",139,21,13.669065,1900,12524.879,90.107043,6720,67.200000


### Avg payout with 10 or more mentions

In [152]:
unique_domains.query('mentions >= 10').sort_values('avg_payout', ascending=False).head()

,domain,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation
98,googleusercontent.com,"[bangking, insomnia, pairmike, riverhead]",51,4,40.784314,2080,31816.348,623.849961,289,72.250000
96,google.com,"[au1nethyb1, cass, chris4210, cnavigato, dedri...",36,19,27.472222,989,14466.093,401.835917,1962,67.655172
31,bitshares.org,"[chris4210, ihashfury, peerplays, tuck-fheman,...",15,6,36.800000,552,5301.370,353.424667,2199,68.718750
119,ipfs.pics,"[ash, kencode, lafona, lovelace, markopaasila,...",28,10,31.535714,883,6479.172,231.399000,1169,58.450000
169,puu.sh,"[au1nethyb1, danknugs, justin, nextgencrypto, ...",31,5,37.032258,1148,6225.434,200.820452,4221,63.954545


### Top 20 websites by author steem power

### Top 20 websites by log of author steem power

## Twitter

In [157]:
domain = 'twitter.com'

In [158]:
unique_urls[unique_urls.domain == domain].head()

,url,authors,mentions,num_authors,avg_votes,total_votes,total_payout,avg_payout,total_reputation,avg_reputation,domain
228,http://www.twitter.com/steemservices,steemservices,1,1,77.0,77,637.063,637.063,130,65.0,twitter.com
472,https://twitter.com/EL/status/732374086580461568,jabbasteem,1,1,1.0,1,0.110,0.110,53,53.0,twitter.com
473,https://twitter.com/SteemServices,steemservices,1,1,21.0,21,68.744,68.744,65,65.0,twitter.com
512,https://www.twitter.com/steemservices,steemservices,1,1,38.0,38,260.800,260.800,130,65.0,twitter.com


In [17]:
domain_authors = url_with_author_data[url_with_author_data.domain == domain].groupby('url', as_index=False)['author'].agg({'authors': lambda x: pd.unique(x), 'author_count': lambda x: len(pd.unique(x))})
domain_authors.sort_values(by='author_count', ascending=False).head(n=20)

,url,author_count,authors
11827,https://twitter.com/bobbyclee/status/865851769...,158,"[clayop, ash, craig-grant, ervin-lemark, kings..."
11810,https://twitter.com/bobbyclee/status/864155469...,60,"[craig-grant, morning, bleepcoin, acidyo, stee..."
3134,https://twitter.com/CCS_Crystal,35,"[taufik123, gie, burmik123, globalbtc, walhall..."
11759,https://twitter.com/blocklancer,35,"[gie, andri-ghani, febieline, yunidita, afansy..."
12678,https://twitter.com/cosscrypto,30,"[blackjincrypto, gie, donaa, tikadara, tempe, ..."
8534,https://twitter.com/SatoshiLite,23,"[joseph, marketingmonk, nphacker, bitcoinmeist..."
1693,https://twitter.com/,22,"[carlos-cabeza, bleepcoin, thecryptofiend, jos..."
11694,https://twitter.com/bitquence,22,"[gie, eviee, vividrange, febieline, alexmedway..."
14894,"https://twitter.com/hashtag/bitcoin?src=hash"">...",22,"[joseph, jupiter00000, juansgalt, jorgefar, aa..."
10745,https://twitter.com/aantonop,21,"[joseph, arcaneinfo, bravenewcoin, joshbreslau..."


In [23]:
twitter_statuses = url_with_author_data[(url_with_author_data.domain == domain) & (url_with_author_data.url.str.contains('/status/'))].groupby('url', as_index=False)['author'].agg({'authors': lambda x: pd.unique(x), 'author_count': lambda x: len(pd.unique(x))})
twitter_statuses.sort_values(by='author_count', ascending=False).head(n=20)

,url,author_count,authors
5119,https://twitter.com/bobbyclee/status/865851769...,158,"[clayop, ash, craig-grant, ervin-lemark, kings..."
5102,https://twitter.com/bobbyclee/status/864155469...,60,"[craig-grant, morning, bleepcoin, acidyo, stee..."
5091,https://twitter.com/bobbyclee/status/862908660...,18,"[morning, kingscrown, acidyo, blakemiles84, th..."
7617,https://twitter.com/realDonaldTrump/status/881...,16,"[leprechaun, shayne, surftrader, fortified, to..."
4371,https://twitter.com/VitalikButerin/status/8791...,11,"[avarice, taufik123, razvanelulmarin, beto91, ..."
4570,https://twitter.com/YourBTCC/status/9082855863...,11,"[marketingmonk, joshbreslauer, penguinpablo, c..."
1872,https://twitter.com/IamNomad/status/9088317644...,9,"[fundurian, malcolmblackman, lukedaniel, jsl1p..."
7227,https://twitter.com/officialmcafee/status/8870...,9,"[cardsanarchy, grossefreiheittv, komersreal, e..."
140,https://mobile.twitter.com/bobbyclee/status/86...,8,"[themanualbot, funderground, swolesome, gingen..."
4750,https://twitter.com/agurevich23/status/9076929...,7,"[crypt0, champagnecrypto, journalducoin, danye..."


In [28]:
twitter_statuses = url_with_author_data[(url_with_author_data.domain == domain) & (url_with_author_data.url.str.count('/') == 3)].groupby('url', as_index=False)['author'].agg({'authors': lambda x: pd.unique(x), 'author_count': lambda x: len(pd.unique(x))})
twitter_statuses.sort_values(by='author_count', ascending=False).head(n=20)

,url,author_count,authors
4800,https://twitter.com/blocklancer,35,"[gie, andri-ghani, febieline, yunidita, afansy..."
1672,https://twitter.com/CCS_Crystal,35,"[taufik123, gie, burmik123, globalbtc, walhall..."
5128,https://twitter.com/cosscrypto,30,"[blackjincrypto, gie, donaa, tikadara, tempe, ..."
3634,https://twitter.com/SatoshiLite,23,"[joseph, marketingmonk, nphacker, bitcoinmeist..."
1114,https://twitter.com/,22,"[carlos-cabeza, bleepcoin, thecryptofiend, jos..."
4773,https://twitter.com/bitquence,22,"[gie, eviee, vividrange, febieline, alexmedway..."
4363,https://twitter.com/aantonop,21,"[joseph, arcaneinfo, bravenewcoin, joshbreslau..."
4059,https://twitter.com/ToneVays,20,"[joseph, marketingmonk, bitcoinmeister, bitcoi..."
4615,https://twitter.com/atlantio,20,"[gie, burmik123, maxim-ananko, gabriel-nilsson..."
5638,https://twitter.com/ex_scudo,20,"[blackjincrypto, gie, febieline, farid, azka, ..."


## Steemit.com

In [29]:
domain = 'steemit.com'

In [30]:
url_with_author_data[url_with_author_data.domain == domain]['url'].value_counts()[0:20]

https://steemit.com/@hr1                                                                                                      8620
https://steemit.com/@dtworker                                                                                                 7773
https://steemit.com/@cryptomonitor                                                                                            7657
https://steemit.com/@blocktrades                                                                                              7415
https://steemit.com/@steemitboard                                                                                             7094
https://steemit.com/@benjamin.still                                                                                           7034
https://steemit.com/~witnesses                                                                                                6302
https://steemit.com/@arama                                                         

In [32]:
domain_authors = url_with_author_data[url_with_author_data.domain == domain].groupby('url', as_index=False)['author'].agg({'author_count': lambda x: len(pd.unique(x))})
domain_authors.sort_values(by='author_count', ascending=False).head(n=20)

,url,author_count
580448,https://steemit.com/~witnesses,438
3710,http://www.steemit.com/~witnesses,373
94497,https://steemit.com,346
316415,https://steemit.com/images/steemit-share.png,224
94503,https://steemit.com/,220
568126,https://steemit.com/video/@heimindanger/introd...,101
316417,"https://steemit.com/images/steemit-share.png""",91
272971,https://steemit.com/faq.html,88
11,http://steemit.com,86
12645,https://img1.steemit.com/0x0/https://i.imgsafe...,83


## Facebook

In [ ]:
domain = 'facebook.com'

In [ ]:
url_with_author_data[url_with_author_data.domain == domain]

In [ ]:
facebook_authors = url_with_author_data[url_with_author_data.domain == domain].groupby('url', as_index=False)['author'].agg({'authors': lambda x: pd.unique(x), 'author_count': lambda x: len(pd.unique(x))})
facebook_authors.sort_values(by='author_count', ascending=False).head(n=20)